# Creating the BBP fi curve datasets

### Load modules

In [2]:
import os
import numpy as np
import pickle

import bluebrain_data_io as bbp_io

result_path = './results/lif_fits/'

### Load experiments

In [3]:
vip_expms = bbp_io.get_vip_expms(merged=False, verbose=False)
som_expms = bbp_io.get_som_expms(merged=False, verbose=False)
pv_expms = bbp_io.get_pv_expms(merged=False, verbose=False)

In [4]:
def get_fi_curves(expms, threshold=0.0, by='spike_count', initial_cut=300):
    i_values = {}
    f_values = {}
    params = {}
    cell_names = []
    
    for expm in expms:
        expm.detectSpikes_cython(threshold=threshold, verbose=False)
        i_values[expm.name], f_values[expm.name] = expm.compute_fi_curve(by=by, initial_cut=initial_cut)
        cell_names.append(expm.name)
    
    cell_type = expm.cell_type
    params['threshold [mV]'] = threshold
    params['fi_computation_method'] = by
    params['initial_cutoff_segment[ms]'] = initial_cut
    
    return {'current_amplitudes':i_values, 'firing_rates':f_values, 'cell_names':cell_names,
            'cell_type':cell_type, 'parameters':params}

In [5]:
def save_fi_data(expm_set):
    fi_data = get_fi_curves(expm_set, threshold=-10.0, by='mean_isi')
    result_path = './results/lif_fits/'
    with open(result_path+'fi_curve_data_' + fi_data['cell_type'] + '.pkl', 'wb') as f:
        pickle.dump(fi_data, f, protocol=2)

In [6]:
save_fi_data(vip_expms)
save_fi_data(som_expms)
save_fi_data(pv_expms)

## Use these methods to load the data

In [8]:
def load_fi_data(cell_type):
    with open(result_path+'fi_curve_data_' + cell_type + '.pkl','rb') as f:
        loaded_data = pickle.load(f)
    return loaded_data

def plot_all_fi_curves(fi_curve_data):
    i_values = fi_curve_data['current_amplitudes']
    f_values = fi_curve_data['firing_rates']
    cell_names = fi_curve_data['cell_names']
    plt.figure()
    for cell in cell_names:
        if not len(i_values[cell]) == 0:
            plt.plot(i_values[cell], f_values[cell], '.-', ms=12, label=cell)
    plt.legend(loc='upper left', bbox_to_anchor=(1.1, 1), ncol=1)
    plt.xlabel('Input current [pA]')
    plt.ylabel('Firing rate [Hz]')
    plt.title(fi_curve_data['cell_type'])
    plt.show()
    
def plot_single_cell_fi_curve(fi_curve_data, cell_name):
    i_values = fi_curve_data['current_amplitudes']
    f_values = fi_curve_data['firing_rates']
    cell_names = fi_curve_data['cell_names']
    plt.figure()
    if not len(i_values[cell_name]) == 0:
        plt.plot(i_values[cell_name], f_values[cell_name], '.-', ms=12, label=cell_name)
    plt.legend(loc='upper left', bbox_to_anchor=(1.1, 1), ncol=1)
    plt.xlabel('Input current [pA]')
    plt.ylabel('Firing rate [Hz]')
    plt.title(fi_curve_data['cell_type'])
    plt.show()
    
def show_all_cell_names(fi_curve_data):
    for cell in fi_curve_data['cell_names']:
        print(cell)
        
def show_parameters(fi_curve_data):
    print(fi_curve_data['parameters'])
        
# Example usage
'''
loaded_data = load_fi_data('vip')
plot_all_fi_curves(loaded_data)
plot_single_cell_fi_curve(loaded_data, 'Cell_A87_single_traces')
show_all_cell_names(loaded_data)
show_parameters(loaded_data)
'''

"\nloaded_data = load_fi_data('vip')\nplot_all_fi_curves(loaded_data)\nplot_single_cell_fi_curve(loaded_data, 'Cell_A87_single_traces')\nshow_all_cell_names(loaded_data)\nshow_parameters(loaded_data)\n"